In [ ]:
#PREREQUISITS

import cv2
import numpy as np 
import matplotlib.pyplot as plt

# Simple image displaying function
def show(img, label=None):
    plt.figure(figsize=(20,20))
    if len(img.shape) == 2:  # Grayscale image
        plt.imshow(img, cmap='gray')
    else:  # Color image
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    if label is not None:
        plt.title(label)
    plt.show()

# Crop function
def crop_image(img, x, y, w, h):
  crop = img[y:y+h, x:x+w]
  return crop

In [ ]:
#Load images
# before = cv2.imread('images/.png') 
# after = cv2.imread('images/.png')

# ADD YOUR CROPPED IMAGE HERE AND RUN ALL CELLS
img = cv2.imread('images/---------.png')

In [ ]:
# PREPROCESS IMAGE

# This block of code is responsible for preprocessing the input image to make it suitable for further analysis. 
# The preprocessing steps include applying a median blur to reduce noise, converting the image to grayscale for easier processing, 
# and applying Otsu's thresholding twice to segment the image into regions of interest. 
# The result of these operations is a binary image where the regions of interest are highlighted.

medianBlur = cv2.medianBlur(img, 11)
gray = cv2.cvtColor(medianBlur, cv2.COLOR_BGR2GRAY)
thresh, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)
masked = cv2.bitwise_and(gray, gray, mask=mask)
thresh, mask2 = cv2.threshold(masked, 0, 255, cv2.THRESH_OTSU)
inverse_mask2 = cv2.bitwise_not(mask2)
inverse_masked = cv2.bitwise_and(gray, gray, mask=mask2)
show(inverse_masked, "Preprocessed Image")

In [ ]:
# QUANTIFY IMAGE

# This block of code is responsible for quantifying the features of interest in the preprocessed image. 
# It first identifies contours in the image, then filters out small contours based on a minimum area threshold. 
# For each remaining contour, it creates a mask and applies it to the original image, 
# then calculates the mean and median pixel values within the masked region. 
# These values are used to calculate a "Linearized RFU Value" for each contour, which is printed out.

contours, _ = cv2.findContours(inverse_masked, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
min_contour_area = 0
filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]
output = masked.copy()
cv2.drawContours(output, filtered_contours, -1, (255, 0, 0), 1)
show(output, "Filtered Contours")

for i, cnt in enumerate(filtered_contours):
    mask = np.zeros_like(gray)
    cv2.drawContours(mask, [cnt], -1, (255), thickness=cv2.FILLED)
    masked = cv2.bitwise_and(gray, gray, mask=mask)
    pixels = masked.flatten()
    pixels = pixels[pixels != 0]
    thresh, mask = cv2.threshold(pixels, 0, 255, cv2.THRESH_OTSU)
    _, realMask = cv2.threshold(masked, thresh, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(realMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        largest_contour_mask = np.zeros_like(gray)
        cv2.drawContours(largest_contour_mask, [largest_contour], -1, (255), thickness=cv2.FILLED)
        masked = cv2.bitwise_and(gray, gray, mask=largest_contour_mask)
        show(masked, "final Mask")
        
        pixels = masked.flatten()
        pixels = pixels[pixels != 0]
        mean_pixels = np.mean(pixels)
        median_pixels = np.median(pixels)

        print(f"Mean of non-zero pixels tube {i+1}: {mean_pixels}")
        print(f"Linearized RFU Value {i+1}: {1.52*10^(-26) * mean_pixels^(13.1)}")